In [15]:
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
import json
import os
import os
import json


def remove_underline(x):
    return x.replace("_", "")

def normalise_pred(pred):
    return pred.strip().split("\n")[0].strip()

class_to_tasks_mapping = {"icl":  ['tweet_hate' ,'tweet_offensive', 'agnews' ,'amazon' ,'dbpedia' ,'sst2','yelp' ],
                          "obqa": [ "squad","hotpotqa"],
                          "obqa_rag": ["nq_obqa", "tq_obqa"],
                          "cbqa": ["tq", "nq"],
                          "memtrap" :["memtrap"]
                          }

class_to_seed_mapping = {"icl": range(42, 58),
                         "obqa": range(42, 46+1),
                         "cbqa": range(42, 46 + 1),
                            "obqa_rag": range(42, 46 + 1),
                         "memtrap":[43]
                         }


In [16]:
BASE_PATH = '/home/aiops/zhuty/tinyllama/scripts/fewshot_eval/outputs/'
commands = []
data = []

size="1b"

# Define the list of iter_name and model_names
# iterations = [
#     {
#         "iter_name": "iter-480000-ckpt-step-60000_hf",
#         "model_names": ['cc_merged_v2_8k', 'intramask_cc_8k', 'cc_merged_v2_8k_intracccont', 'cc_8k', 'adamask_cc_merged_v2_8k', 'intramask_cc_merged_v2_8k']
#     },
#     {
#         "iter_name": "iter-380000-ckpt-step-47500_hf",
#         "model_names": ['cc_merged_v2_8k', 'intramask_cc_8k', 'cc_merged_v2_8k_intracccont', 'cc_8k', 'adamask_cc_merged_v2_8k', 'intramask_cc_merged_v2_8k']
#     },
#     {
#         "iter_name": "iter-600000-ckpt-step-75000_hf",
#         "model_names": ['cc_merged_v2_8k', 'intramask_cc_8k', 'cc_merged_v2_8k_intracccont', 'cc_8k', 'cc_merged_v1_8k', 'adamask_cc_merged_v2_8k', 'intramask_cc_merged_v2_8k']
#     }
# ]

iterations = []
model_names = ['cc_merged_v2_8k' ,'cc_8k']
for step in range(5000, 80000, 5000):
    iter_name = f'iter-{step*8:06}-ckpt-step-{step}_hf'
    iterations.append({"iter_name": iter_name, "model_names": model_names})

# Define the list of TASK_CLASS, shot_nums, and rag_num_docs_lst
tasks = [
    {
        "TASK_CLASS": "obqa",
        "shot_nums": [4],
        "rag_num_docs_lst": [""]
    },
    {
        "TASK_CLASS": "icl",
        "shot_nums": [24, 48],
        "rag_num_docs_lst": [""]
    },
    {
        "TASK_CLASS": "memtrap",
        "shot_nums": [0],
        "rag_num_docs_lst": [""]
    },
    {
        "TASK_CLASS": "cbqa",
        "shot_nums": [12, 24],
        "rag_num_docs_lst": [""]
    },
    {
        "TASK_CLASS": "obqa_rag",
        "shot_nums": [2],
        "rag_num_docs_lst": [1, 3, 5, 10]
    }
]

selected_task = "obqa"
tasks = [task for task in tasks if task["TASK_CLASS"] == selected_task]

# Loop through each set of iter_name and model_names
for iteration in iterations:
    iter_name = iteration["iter_name"]
    model_names = iteration["model_names"]

    print(f"Processing {iter_name} with models:")
    for model_name in model_names:
        print(f" - {model_name}")
    # Loop through each set of TASK_CLASS, shot_nums, and rag_num_docs_lst
    for task in tasks:
        TASK_CLASS = task["TASK_CLASS"]
        shot_nums = task["shot_nums"]
        rag_num_docs_lst = task["rag_num_docs_lst"]



        for task_name in  class_to_tasks_mapping[TASK_CLASS]:
            # for shot_num in [1, 3, 5, 10, 20, 30, 48]:
            # for shot_num in [3,5,10,20,30,48]:
            # for shot_num in [24, 48]:
            for shot_num in shot_nums:
                for rag_num_docs in rag_num_docs_lst:
                    if task_name == "yelp" and shot_num == 48:
                       continue
                    for model_name in model_names:

                        if '8k' not in model_name and 'cc' in model_name:
                            model_name = f"{model_name}_8k"
                    # for model_name in ['bstilm']:
                        for seed_num in class_to_seed_mapping[TASK_CLASS]:
                        # for seed_num in [42]:
                            full_name = f"tiny_LLaMA_{size}_8k_{model_name}-{iter_name}"
                            # full_name = "TinyLlama-1.1B-step-50K-105b"
                            if TASK_CLASS == "obqa_rag":
                                result_path = os.path.join(BASE_PATH,full_name, f'{task_name}_{shot_num}_{seed_num}_{rag_num_docs}.json')
                            else:
                                result_path = os.path.join(BASE_PATH,full_name, f'{task_name}_{shot_num}_{seed_num}.json')
                            if not os.path.exists(result_path):
                                # print(f"File {result_path} does not exist")
                                # model_path = f"/home/aiops/zhuty/tinyllama/out/tiny_LLaMA_{size}_8k_{model_name}/{iter_name}"
                                model_path = f"tyzhu/tiny_LLaMA_{size}_8k_{model_name}_{iter_name}"
                                if model_name in ['BM25Chunk', 'IntraDoc', 'UniChunk', 'MixChunk'] :
                                    model_path = f"yuzhaouoe/{model_name}"


                                # check if config file exists
                                assert os.path.exists(f"{model_path}/config.json") or 'tyzhu/' in model_path or 'yuzhao' in model_path, "Missing config file for {}, convert it with \n {}".format(model_path, f"bash convert_to_hf_general.sh /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_{size}_8k_{model_name} {iter_name.strip('_hf')}")

                                if TASK_CLASS == "obqa_rag":
                                    assert str(rag_num_docs) != ""
                                else:
                                    assert str(rag_num_docs) == ""
                                jobname = f"eval{size.lower()}{remove_underline(task_name)}{remove_underline(model_name)}{seed_num}"[:40]
                                # curr = f""" sailctl job create {jobname} -g 1  --debug --image asia-docker.pkg.dev/sail-tpu-02/git-insea-io/common/image_registry/liuqian/tinyllama:v9   --command-line-args  ' bash /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/ ; bash single_task.sh {model_path} {task_name} {seed_num} {shot_num} {rag_num_docs}' {high_string} """
                                curr = (jobname, model_path, task_name, f"bash single_task.sh {model_path} {task_name} {seed_num} {shot_num} {rag_num_docs} ")

                                commands.append(curr)

                                continue
                            else:
                                # print("Existing", result_path)
                                result = json.load(open(result_path, 'r'))
                                if TASK_CLASS == "obqa_rag":
                                    result['num_docs'] = rag_num_docs
                                data.append(result)



Processing iter-040000-ckpt-step-5000_hf with models:
 - cc_merged_v2_8k
 - cc_8k
Processing iter-080000-ckpt-step-10000_hf with models:
 - cc_merged_v2_8k
 - cc_8k
Processing iter-120000-ckpt-step-15000_hf with models:
 - cc_merged_v2_8k
 - cc_8k
Processing iter-160000-ckpt-step-20000_hf with models:
 - cc_merged_v2_8k
 - cc_8k
Processing iter-200000-ckpt-step-25000_hf with models:
 - cc_merged_v2_8k
 - cc_8k
Processing iter-240000-ckpt-step-30000_hf with models:
 - cc_merged_v2_8k
 - cc_8k
Processing iter-280000-ckpt-step-35000_hf with models:
 - cc_merged_v2_8k
 - cc_8k
Processing iter-320000-ckpt-step-40000_hf with models:
 - cc_merged_v2_8k
 - cc_8k
Processing iter-360000-ckpt-step-45000_hf with models:
 - cc_merged_v2_8k
 - cc_8k
Processing iter-400000-ckpt-step-50000_hf with models:
 - cc_merged_v2_8k
 - cc_8k
Processing iter-440000-ckpt-step-55000_hf with models:
 - cc_merged_v2_8k
 - cc_8k
Processing iter-480000-ckpt-step-60000_hf with models:
 - cc_merged_v2_8k
 - cc_8k
Proce

In [19]:
assert len(commands) == 0, "There are missing files for the following commands: \n" + "\n".join([f"{c[0]}: {c[1]}" for c in commands])

AssertionError: There are missing files for the following commands: 
eval1bsquadccmergedv28k42: tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-040000-ckpt-step-5000_hf
eval1bsquadccmergedv28k43: tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-040000-ckpt-step-5000_hf
eval1bsquadccmergedv28k44: tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-040000-ckpt-step-5000_hf
eval1bsquadccmergedv28k45: tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-040000-ckpt-step-5000_hf
eval1bsquadccmergedv28k46: tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-040000-ckpt-step-5000_hf
eval1bsquadcc8k42: tyzhu/tiny_LLaMA_1b_8k_cc_8k_iter-040000-ckpt-step-5000_hf
eval1bsquadcc8k43: tyzhu/tiny_LLaMA_1b_8k_cc_8k_iter-040000-ckpt-step-5000_hf
eval1bsquadcc8k44: tyzhu/tiny_LLaMA_1b_8k_cc_8k_iter-040000-ckpt-step-5000_hf
eval1bsquadcc8k45: tyzhu/tiny_LLaMA_1b_8k_cc_8k_iter-040000-ckpt-step-5000_hf
eval1bsquadcc8k46: tyzhu/tiny_LLaMA_1b_8k_cc_8k_iter-040000-ckpt-step-5000_hf
eval1bhotpotqaccmergedv28k42: tyzhu/tiny_LLaMA_1b_8k_cc_merge

In [8]:
df = pd.DataFrame(data)

# ensure that no score is zero
assert len(df[df['score'] == 0]) == 0

In [9]:
def extract_step(x):
    # strip the _hf
    x = x[:-3]
    return int(x.split('-')[-1])

def extract_model_name(x):
    """
    Sample input: /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_1b_8k_cc_8k/iter-200000-ckpt-step-25000_hf
    Sample output: tiny_LLaMA_1b_8k
    Args:
        x:
    Returns:

    """
    if 'TinyLlama/TinyLlama-1.1B-step-50K-105b' in x:
        return 'tinyllama-1.1b-50k-105b'
    if '/home/aiops/zhuty' in x:
        folder_name = x.split('/')[-2] # tiny_LLaMA_1b_8k_cc_8k
        return ("_").join(folder_name.split('_')[:4]) # tiny_LLaMA_1b_8k
    else:
        # sample model name: tyzhu/tiny_LLaMA_120M_8k_cc_8k_iter-240000-ckpt-step-60000_hf
        return x.strip("tyzhu/").split("_cc")[0]

def extract_pretrain_dataset(x):
    """
    Sample input: /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_1b_8k_cc_8k/iter-200000-ckpt-step-25000_hf
    Sample output: cc_8k
    Args:
        x:
    Returns:

    """
    if '/home/aiops/zhuty' in x:
        folder_name = x.split('/')[-2] # tiny_LLaMA_1b_8k_cc_8k
        return ("_").join(folder_name.split('_')[4:]) # cc_8k
    else:
        # sample model name: tyzhu/tiny_LLaMA_120M_8k_cc_8k_iter-240000-ckpt-step-60000_hf
        return x.strip("tyzhu/").split("_8k_")[1]


In [10]:
df

,task,n_shot,seed,model,score
0,tweet_hate,24,42,tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-04...,45.084175
1,tweet_hate,24,43,tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-04...,40.909091
2,tweet_hate,24,44,tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-04...,56.700337
3,tweet_hate,24,45,tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-04...,49.932660
4,tweet_hate,24,46,tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-04...,44.276094
...,...,...,...,...,...
6235,yelp,24,53,tyzhu/tiny_LLaMA_1b_8k_cc_8k_iter-600000-ckpt-...,86.920000
6236,yelp,24,54,tyzhu/tiny_LLaMA_1b_8k_cc_8k_iter-600000-ckpt-...,85.740000
6237,yelp,24,55,tyzhu/tiny_LLaMA_1b_8k_cc_8k_iter-600000-ckpt-...,90.560000
6238,yelp,24,56,tyzhu/tiny_LLaMA_1b_8k_cc_8k_iter-600000-ckpt-...,92.200000


In [14]:
import re

def extract_model_and_steps(model_name):
    """
    Extracts the model name and step count from the model string.

    Parameters:
    model_name (str): The full model string.

    Returns:
    tuple: A tuple containing the model name and step count.
    """
    model_pattern = re.compile(r'tyzhu/tiny_LLaMA_1b_8k_(.+?)_iter-\d+-ckpt-step-(\d+)_hf')
    match = model_pattern.search(model_name)
    if match:
        return match.group(1), int(match.group(2))
    return None, None

# Apply the function to extract model names
extracted_values = df['model'].apply(extract_model_and_steps)
df['model_name'] = extracted_values.apply(lambda x: x[0] if x else None)
df['steps'] = extracted_values.apply(lambda x: x[1] if x else None)

# Group the data by task, n_shot, and model and calculate the mean and std deviation
grouped_data = df.groupby(['task', 'n_shot', 'model_name', 'steps'])['score'].agg(['mean', 'std']).reset_index()

# Display the processed data
grouped_data


,task,n_shot,model_name,steps,mean,std
0,agnews,24,cc_8k,5000,50.938322,11.328001
1,agnews,24,cc_8k,10000,57.884868,7.639003
2,agnews,24,cc_8k,15000,64.957237,7.627914
3,agnews,24,cc_8k,20000,67.393914,6.313641
4,agnews,24,cc_8k,25000,68.665296,6.989413
...,...,...,...,...,...,...
385,yelp,24,cc_merged_v2_8k,55000,89.588750,2.670450
386,yelp,24,cc_merged_v2_8k,60000,80.386250,11.988452
387,yelp,24,cc_merged_v2_8k,65000,91.262500,1.637606
388,yelp,24,cc_merged_v2_8k,70000,91.310000,2.002319
